# DB2 RESTful Jupyter Notebook Extensions
Version: 2019-09-04

- 2018-06-01: Initial Release
- 2019-09-04: Moved from clauses to avoid error on forward referencing imports

This code is imported as a Jupyter notebook extension in any notebooks you create with DB2 code in it. Place the following line of code in any notebook that you want to use these commands with:
<pre>
&#37;run db2re.ipynb
</pre>

This code defines a Jupyter/Python magic command called `%sql` which allows you to execute DB2 specific calls to 
the database. There are other packages available for manipulating databases, but this one has been specifically
designed for demonstrating a number of the SQL features available in DB2 using RESTful calls. Note that this code will **only** work for a Db2 on Cloud instance and is not supported for connecting to a standard Db2 server. To use this code on with a Db2 server run the **`db2.ipynb`** command instead.

There are two ways of executing the `%sql` command. A single line SQL statement would use the
line format of the magic command:
<pre>
%sql SELECT * FROM EMPLOYEE
</pre>

If you have a large block of sql then you would place the %%sql command at the beginning of the block and then
place the SQL statements into the remainder of the block. Using this form of the `%%sql` statement means that the
notebook cell can only contain SQL and no other statements.
<pre>
%%sql
SELECT * FROM EMPLOYEE
ORDER BY LASTNAME
</pre>

You can have multiple lines in the SQL block (`%%sql`). The default SQL delimiter is the semi-column (`;`).
If you have scripts (triggers, procedures, functions) that use the semi-colon as part of the script, you 
will need to use the `-d` option to change the delimiter to an at @ sign. 
<pre>
%%sql -d
SELECT * FROM EMPLOYEE
@
CREATE PROCEDURE ...
@
</pre>

The `%sql` command allows most DB2 commands to execute and has a special version of the CONNECT statement. 

The CONNECT command has the following format:

```
%sql CONNECT CREDENTIALS <variable>
```

The `CREDENTIALS` can be supplied as a variable in which case the contents will be used to create an access token to the database and if successful, the variable will be saved to disk for future use. If you create another notebook and use the identical syntax, if the variable is not defined, the contents on disk will be used as the credentials. You should assign the credentials to a variable that represents the database (or schema) that you are communicating with. Using familiar names makes it easier to remember the credentials when connecting. 

For example, if the credentials are assigned to a variable called sample:
```python
sample = \
{
... credentials ...
}
```
Then the connect statement would be:
```
%sql connect credentials sample
```
If you open up a new notebook, you will not have to create the sample variable with the credentials in it. The `%sql connect` command will determine that the variable does not exist and will read the contents of the file with the same name `sample.pickle` and retrieve the credentials that way.

In addition to the -d option, there are a number different options that you can specify at the beginning of 
the SQL. Some of these options can only be used for single line commands (**`%sql`**) while others can be used for both forms of the command. 

### Options for **`%sql`** commands only

   - `-r` - Return the result set as an array of values instead of a dataframe
   - `-sampledata` - Create and load the EMPLOYEE and DEPARTMENT sample tables
   
The `-sampledata` flag will create the sample tables and not process any SQL statements that may be in your `%sql` block. The `-r` flag is meant for returning an answer set back to a variable. For intance:

```python
results = %sql -r select * from employee
```

You can only run SQL statements within an `%%sql` block, so the statement above would not work. If you have a large SQL statement that you want to assign to a variable, you can use one of two approaches. The first option is to place the contents of your SQL statement into a variable:

```python
somesql = """
SELECT * 
FROM EMPLOYEE
"""
```

Then you can issue the `%sql` call using the form:

```python
results = %sql -r {somesql}
```

The Python interpreter will take the contents of the variable `somesql` and place it into the command. The other option is to use Python line continuation characters. To extend the command beyond one line, place a `\` at the end of each line:

```python
results = %sql -r \
SELECT * \
FROM EMPLOYEE 
```
   
### Options for **`%%sql`** and **`%sql`** commands

   - `-d` - Delimiter: Change SQL delimiter to "`@`" from "`;`"
   - `-q` - Quiet: Quiet results - no messages returned from the function
   - `-a` - All: Display all rows in answer set and do not limit the rows displayed
   - `-e` - Any macro expansions are displayed in an output box 
   
### Options for **`%%sql`** and **`%sql`** Commands that Terminate Blocks

These options will work with both forms of the `%sql` magic command, but will only execute on one SQL statement. What this means is that in a `%%sql` block, only the first SQL statement will execute and have the flag applied to it. In addition, any SQL in a `%%sql` block that generates output will terminate the block.

   - `-j` - JSON: Create a pretty JSON representation. Only the first column is formatted
   - `-pb` - Plot Bar: Plot the results as a bar chart
   - `-pl` - Plot Line: Plot the results as a line chart
   - `-pp` - Plot Pie: Plot the results as a pie chart
   - `-i`  - Interactive plot mode (Only available if you install Pixiedust)


<p>
You can pass python variables to the `%sql` or `%%sql` command by using a colon `:` before the name of the variable. Note that you will need to place proper punctuation around the variable in the event the
SQL command requires it. For instance, the following example will find employee '000010' in the EMPLOYEE table.
<pre>
empno = '000010'
%sql SELECT LASTNAME FROM EMPLOYEE WHERE EMPNO=':empno'
</pre>

### Db2 Jupyter Extensions
This section of code has the import statements and global variables defined for the remainder of the functions.

In [ ]:
#
# Set up Jupyter MAGIC commands "sql". 
# %sql will return results from a DB2 select statement or execute a DB2 command
#
# IBM 2018: George Baklarz
# Version 2018-06-25 RESTful Version
#

# Override the name of display, HTML, and Image in the event you plan to use the pixiedust library for
# rendering graphics.

from __future__ import print_function
from IPython.display import HTML as pHTML, Image as pImage, display as pdisplay, Javascript as Javascript
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)
from pixiedust.display import *
from pixiedust.utils.shellAccess import ShellAccess

import pandas
import json
import matplotlib
import matplotlib.pyplot as plt
import os
import pickle
import time
import sys
import re
import warnings
import requests
import qgrid
import pixiedust

warnings.filterwarnings("ignore")

# Python Hack for Input between 2 and 3

try: 
    input = raw_input 
except NameError: 
    pass 

_settings = {
     "id"       : "",
     "token"    : "",
     "connected": False,
     "api"      : "/dbapi/v3"
}

_display = {
    'fullWidthRows': True,
    'syncColumnCellResize': True,
    'forceFitColumns': False,
    'defaultColumnWidth': 150,
    'rowHeight': 28,
    'enableColumnReorder': False,
    'enableTextSelectionOnCells': True,
    'editable': False,
    'autoEdit': False,
    'explicitInitialization': True,
    'maxVisibleRows': 15,
    'minVisibleRows': 8,
    'sortable': True,
    'filterable': False,
    'highlightSelectedCell': False,
    'highlightSelectedRow': True
}

qgrid.set_defaults(grid_options=_display)
pandas.options.display.max_rows = 10

# Global information 

_vars = {}
_macros = {}
_flags = []

# Db2 Error Messages and Codes
sqlcode = 0
sqlstate = "0"
sqlerror = ""

### SQL Help

The calling format of this routine is:

```
sqlhelp()
```

This code displays help related to the %sql magic command. This help is displayed when you issue a %sql or %%sql command by itself, or use the %sql -h flag.

In [ ]:
def sqlhelp():
    
    sd = '<td style="text-align:left;">'
    ed = '</td>'
    sh = '<th style="text-align:left;">'
    eh = '</th>'
    sr = '<tr>'
    er = '</tr>'
    
    helpSQL = """
       <h3>SQL Options</h3> 
       <p>The following options are available as part of a SQL statement. The options are always preceded with a
       minus sign (i.e. -q).
       <table>
        {sr}
           {sh}Option{eh}
           {sh}Description{eh}
        {er}
        {sr}
           {sd}-a{ed}
           {sd}Return all rows in answer set and do not limit display{ed}
        {er}       
        {sr}
          {sd}-d{ed}
          {sd}Change SQL delimiter to "@" from ";"{ed}
        {er}
        {sr}
          {sd}-h{ed}
          {sd}Display %sql help information.{ed}
        {er}         
        {sr}
          {sd}-j{ed}
          {sd}Create a pretty JSON representation. Only the first column is formatted{ed}
        {er} 
        {sr}
          {sd}-pb{ed}
          {sd}Plot the results as a bar chart{ed}
        {er}
        {sr}
          {sd}-pl{ed}
          {sd}Plot the results as a line chart{ed}
        {er}
        {sr}
          {sd}-pp{ed}
          {sd}Plot Pie: Plot the results as a pie chart{ed}
        {er}        
        {sr}
          {sd}-q{ed}
          {sd}Quiet results - no messages returned from the function{ed}
        {er}
        {sr}  
          {sd}-r{ed}
          {sd}Return the result set as an array of values{ed}
        {er}
        {sr}
          {sd}-sampledata{ed}
          {sd}Create and load the EMPLOYEE and DEPARTMENT tables{ed}
        {er}        
       </table>
       """
    
    helpSQL = helpSQL.format(**locals())
        
    pdisplay(pHTML(helpSQL))

### Connection Help

The calling format of this routine is:

```
connected_help()
```

This code displays help related to the CONNECT command. This code is displayed when you issue a %sql CONNECT command with no arguments or you are running a SQL statement and there isn't any connection to a database yet.

In [ ]:
def connected_help():
    
    sd = '<td style="text-align:left;">'
    ed = '</td>'
    sh = '<th style="text-align:left;">'
    eh = '</th>'
    sr = '<tr>'
    er = '</tr>'
        
    helpConnect = """
       <h3>Connecting to DB2</h3> 
       <p>The CONNECT command has the following format:
       <p>
       <pre>
       %sql CONNECT CREDENTIALS &lt;variable&gt 
       </pre>
       The CREDENTIALS can be supplied as a variable in which case the contents will be used to 
       create an access token to the database and if successful, the variable will be saved to disk 
       for future use. If you create another notebook and use the identical syntax, if the variable 
       is not defined, the contents on disk will be used as the credentials. You should assign the 
       credentials to a variable that represents the database (or schema) that you are communicating with. 
       Using familiar names makes it easier to remember the credentials when connecting. 
       """
    
    helpConnect = helpConnect.format(**locals())
        
    pdisplay(pHTML(helpConnect))

### Connect Syntax Parser
The parseConnect routine is used to parse the CONNECT command that the user issued within the `%sql` command. The format of the command is:

```
parseConnect(inSQL)
```

The inSQL string contains the CONNECT keyword with some additional parameters. The format of the CONNECT command is one of:

```
%sql CONNECT CREDENTIALS variable
```

In [ ]:
# Parse the CONNECT statement and execute if possible 

def parseConnect(inSQL):
    
    global _settings

    cParms = inSQL.split()
    cnt = 0

    while cnt < len(cParms):
        if cParms[cnt].upper() == 'CREDENTIALS':
            if cnt+1 < len(cParms):
                credentials = cParms[cnt+1]
                tempid = getContents(credentials)
                if (tempid == None):
                    fname = credentials + ".pickle"
                    try:
                        with open(fname,'rb') as f: 
                            id = pickle.load(f) 
                    except:
                        errormsg("Unable to find Credentials variable or file.")
                        return
                else:
                    id = json.loads(tempid)
                    
                _settings["id"] = id
                token = get_token()                

                if (token == None): 
                    errormsg("Credentials variable or file is invalid.")
                    return                        
                try:
                    fname = credentials + ".pickle"
                    with open(fname,'wb') as f:
                        pickle.dump(id,f)
                except:
                    errormsg("Failed trying to write Db2 RESTful Credentials.")
                    return
                
                _settings["token"] = token 
                
                cnt += 2
                
            else:
                errormsg("No Credentials name supplied")
                return
            
        else:
            cnt += 1
            
    success("Command completed.")

### Request Access Token Routine
In order to communicate with Db2 on Cloud with RESTful calls, you must have a valid access token. This token is valid for a short period of time (approximately an hour) so you may need to revalidate an access token during your session with Db2. There are two approaches that can be used when using the Db2 extensions:

* Use `AUTOCONNECT ON` (default) which will generate an access token for every RESTful call to Db2. This will eliminate the need to worry about recreating an access token, but the downside is that there is the overhead of an additional call to retrieve a valid access token.

* Use `AUTOCONNECT OFF` which will generate an access token on the first connect, but it will not renew it until you issue another CONNECT.

In the event of an error, the routine will print the error message and return `None`. Note that the API library needs to be supplied to the routine since it can be different between releases.

The parameters to the `get_token` call are:
* db2id - The JSON structure that contains the credentials of the user
* api - The API library name in the format /api/...

In [ ]:
def get_token():
    
    import requests
    
    global _settings
    
    _settings['connected'] == False
    
    # Set up structures required for the RESTful call

    db2id = _settings["id"]
    api = _settings["api"]
    
    service = "/auth/tokens"
    host = db2id['https_url'] + api
    userinfo = {
        "userid" : db2id['username'],
        "password" : db2id['password']
    }
    
    # Call the RESTful service
    
    try:
        r = requests.post(host + service, json=userinfo)
    except Exception as err:
        print("RESTful call failed. Detailed information follows.")
        print(err)
        return None
    
    # Check for Invalid credentials
    if (r.status_code == 401): # There was an error with the authentication
        print("RESTful called failed.")
        message = r.json()['message']
        print(message)
        return None
    
    # Check for anything other than 200/401
    if (r.status_code != 200): # Some other failure
        print("RESTful called failed. Detailed information follows.")
        print(r.json())
        return None
    
    # Retrieve the access token
    try:
        access_token = r.json()['token']
    except:
        print("RESTful call did not return an access token.")
        print(r.json())
        return None

    _settings['connected'] = True
    return access_token

### Error and Message Functions

There are three types of messages that are thrown by the %db2 magic command. The first routine will print out a success message with no special formatting:

```
success(message)
```

The second message is used for displaying an error message that is not associated with a SQL error. This type of error message is surrounded with a red box to highlight the problem. Note that the success message has code that has been commented out that could also show a successful return code with a green box. 

```
errormsg(message)
```

The final error message is based on an error occuring in the SQL code that was executed. This code will parse the message returned from the ibm_db interface and parse it to return only the error message portion (and not all of the wrapper code from the driver).

```
db2_error(quiet)
```

The quiet flag is passed to the db2_error routine so that messages can be suppressed if the user wishes to ignore them with the `-q` flag. A good example of this is dropping a table that does not exist. We know that an error will be thrown so we can ignore it. The information that the db2_error routine gets is from the stmt_errormsg() function from within the ibm_db driver. The db2_error function should only be called after a SQL failure otherwise there will be no diagnostic information returned from stmt_errormsg().

In [ ]:
def db2_error(quiet):
    
    if quiet == True: return

    html = '<p><p style="border:2px; border-style:solid; border-color:#FF0000; background-color:#ffe6e6; padding: 1em;">'

    errmsg = ibm_db.stmt_errormsg().replace('\r',' ')
    errmsg = errmsg[errmsg.rfind("]")+1:].strip()
    pdisplay(pHTML(html+errmsg+"</p>"))
    
# Print out an error message

def errormsg(message):
    
    global sqlerror, sqlcode, sqlstate
    
    msg_start = message.find("SQLSTATE=")
    if (msg_start != -1):
        msg_end = message.find(",",msg_start)
        if (msg_end == -1):
            msg_end = len(errmsg)
        sqlstate = message[msg_start+9:msg_end]
    else:
        sqlstate = "0"
    
    msg_start = message.find("SQLCODE=")
    if (msg_start != -1):
        msg_end = message.find(",",msg_start)
        if (msg_end == -1):
            msg_end = len(errmsg)
        sqlcode = message[msg_start+8:msg_end]
        try:
            sqlcode = int(sqlcode)
        except:
            pass
    else:
        sqlcode = 0
        
    if (sqlstate != "0" and sqlcode != 0) : sqlerror = message
    
    if flag("-q") == True: return
    
    if (message != ""):
        html = '<p><p style="border:2px; border-style:solid; border-color:#FF0000; background-color:#ffe6e6; padding: 1em;">'
        pdisplay(pHTML(html + message + "</p>"))     
    
def success(message):
    
    if (message != ""):
        print(message)        
    return   

def debug(message):
    
    if (message != ""):

        lines = message.split('\n')
        msg = ""
        indent = 0
        for line in lines:
            delta = line.count("(") - line.count(")")
            if (msg == ""):
                msg = line
                indent = indent + delta
            else:
                if (delta < 0): indent = indent + delta
                msg = msg + "<br>" + "&nbsp;" * (indent*2) + line
                if (delta > 0): indent = indent + delta    

            if (indent < 0): indent = 0
                    
        html = '<p><p style="font-family: monospace; border:2px; border-style:solid; border-color:#008000; background-color:#e6ffe6; padding: 1em;">'
        pdisplay(pHTML(html + msg + "</p>"))
        
    return 

## Macro Processor
This code is used for shortening some SQL that is used when dealing with JSON objects. There are five routines that deal with macros. 

- subMacro is used to find the macro calls in a string. All macros are sent to parseMacro for checking.
- parseMacro will parse the macro call and then run the macro code by calling runMacro.
- runMacro will evaluate the macro and return the string to the parse
- subvars is used to track the variables used as part of a macro call.
- setMacro is used to catalog a macro

### Set Macro
This code will catalog a macro call.

In [ ]:
def setMacro(inSQL,parms):
    
    global _macros
    
    names = parms.split()
    if (len(names) < 2):
        errormsg("No macro name supplied.")
        return None
    
    _macros[names[1]] = inSQL
    return

### Search Macro
This code will go through an entire string and replace the macro calls with the proper values.

In [ ]:
def searchMacro(instr):
    
    import copy
    
    global _vars
    quoteChar = ""
    inQuote = False
    inMacro = False
    inArgs = False
    tokens = []
    token = ''
    parms = []
    
    _tempvars = {}
    for var in _vars:
        if (var[0] == "$"): _tempvars[var] = _vars[var]
            
    _vars.clear()
    _vars = copy.copy(_tempvars)
    
    for ch in instr:
        if (inMacro == True):
            token = token + ch
            if (inQuote == True):
                if (quoteChar == ch):
                    inQuote = False
            elif (ch == "\"" or ch == "\'" or ch == "["): # Do we have a quote
                if (ch == "["):
                    quoteChar = "]"
                else:
                    quoteChar = ch
                inQuote = True
            elif (ch == "("):
                inArgs = True
            elif (ch == ")" and inArgs == True):
                inMacro = False
                inArgs = False
                result = subMacro(token)
                if (result == None):
                    return None
                tokens.append(result)
                token = ""
            elif (ch in [" ","\n"] and inArgs == False):
            # elif (ch == " " and inArgs == False):
                inMacro = False
                result = subMacro(token)
                if (result == None):
                    return None
                tokens.append(result)
                token = ch
        else:
            if (inQuote == True):
                token = token + ch
                if (quoteChar == ch):
                    inQuote = False
            elif (ch == "\"" or ch == "\'"): # Do we have a quote
                token = token + ch
                quoteChar = ch
                inQuote = True 
            elif (ch == "%"):
                if (token != ""):
                    tokens.append(token)
                token = ""
                inMacro = True
            else:
                token = token + ch
                
    if (inMacro == True):
        result = subMacro(token)
        if (result == None):
            return None
        tokens.append(result)
        token = ""
       
    if (token != ""):
        tokens.append(token)
        
    sql = ""
    for s in tokens:
        sql = sql + s
        
    if flag("-e"): debug(sql)
        
    return(sql)

## Substitute Macro
This code will parse and run the macro and return a string to be inserted into the SQL.

In [ ]:
def subMacro(macro):
    
    global _macros, _vars
    
    parms = []
    _vars["argc"] = "0"
    
    macro_name, args = parseCallArgs(macro)
    if (macro_name in _macros):
        for arg in args:
            if (arg[0] == "null"):
                parms.append(arg[1])
            else:
                _vars[arg[0]] = arg[1]
    else:
        print("Macro: "+ macro_name + " not defined.")
        return None 
        
    _vars['argc'] = str(len(parms))   
    result = runMacro(_macros[macro_name],parms)
    return(result)

### Parse Call Arguments
This code will parse a macro or SQL call #name(parm1,...) and return the name and the parameters in the call.

In [ ]:
def parseCallArgs(macro):
    
    quoteChar = ""
    inQuote = False
    inParm = False
    name = ""
    parms = []
    parm = ''
    
    sql = macro
    
    for ch in macro:
        if (inParm == False):
            if (ch in ["("," ","\n"]): 
                inParm = True
            else:
                name = name + ch
        else:
            if (inQuote == True):
                if (ch == quoteChar):
                    inQuote = False  
                    #if (quoteChar == "]"):
                    #    parm = parm + "'"
                else:
                    parm = parm + ch
            elif (ch in ("\"","\'","[")): # Do we have a quote
                if (ch == "["):
                   # parm = parm + "'"
                    quoteChar = "]"
                else:
                    quoteChar = ch
                inQuote = True
            elif (ch == ")"):
                if (parm != ""):
                    parm_name, parm_value = splitassign(parm)
                    parms.append([parm_name,parm_value])
                parm = ""
                break
            elif (ch == ","):
                if (parm != ""):
                    parm_name, parm_value = splitassign(parm)
                    parms.append([parm_name,parm_value])                  
                else:
                    parms.append(["null","null"])
                    
                parm = ""

            else:
                parm = parm + ch
                
    if (inParm == True):
        if (parm != ""):
            parm_name, parm_value = splitassign(parm)
            parms.append([parm_name,parm_value])      
       
    return(name,parms)

### Run Macro
This code will execute the body of the macro and return the results for that macro call.

In [ ]:
def runMacro(script,parms):
    
    global _vars
    
    result = ""
    runIT = True 
    code = script.split("\n")
    level = 0
    runlevel = [True,False,False,False,False,False,False,False,False,False]
    ifcount = 0
    
    for i in range(0,len(parms)):
        vstr = str(i+1)
        _vars[vstr] = parms[i]
    
    for line in code:
        line = line.strip()
        if (line == "" or line == "\n"): continue
        args = parseArgs(line)     # Get all of the arguments
 
        if (args[0] == "if"):
            ifcount = ifcount + 1
            if (runlevel[level] == False): # You can't execute this statement
                continue
            level = level + 1    
            if (len(args) < 4):
                print("Macro: Incorrect number of arguments for the if clause.")
                return None
            arg1 = args[1]
            arg2 = args[3]
            if (len(arg2) > 2):
                ch1 = arg2[0]
                ch2 = arg2[-1:]
                if (ch1 in ['"',"'"] and ch1 == ch2):
                    arg2 = arg2[1:-1].strip()
               
            op   = args[2]

            if (op in ["=","=="]):
                if (arg1 == arg2):
                    runlevel[level] = True
                else:
                    runlevel[level] = False                
            elif (op in ["<=","=<"]):
                if (arg1 <= arg2):
                    runlevel[level] = True
                else:
                    runlevel[level] = False                
            elif (op in [">=","=>"]):                    
                if (arg1 >= arg2):
                    runlevel[level] = True
                else:
                    runlevel[level] = False                                       
            elif (op in ["<>","!="]):                    
                if (arg1 != arg2):
                    runlevel[level] = True
                else:
                    runlevel[level] = False  
            elif (op in ["<"]):
                if (arg1 < arg2):
                    runlevel[level] = True
                else:
                    runlevel[level] = False                
            elif (op in [">"]):
                if (arg1 > arg2):
                    runlevel[level] = True
                else:
                    runlevel[level] = False                
            else:
                print("Macro: Unknown comparison operator in the if statement:" + op)

                continue

        elif (args[0] == "echo" and runlevel[level] == True):
            msg = ""
            for msgline in args[1:]:
                if (msg == ""):
                    msg = subvars(msgline)
                else:
                    msg = msg + " " + subvars(msgline)
            if (msg != ""): debug(msg)

        elif (args[0] == "exit" and runlevel[level] == True):
            return None

        elif (args[0] == "var" and runlevel[level] == True):
            value = ""
            for val in args[2:]:
                if (value == ""):
                    value = subvars(val)
                else:
                    value = value + " " + subvars(val)
            value.strip()
            _vars[args[1]] = value 

        elif (args[0] == 'else'):

            if (ifcount == level):
                runlevel[level] = not runlevel[level]

        elif (args[0] == "endif"):
            ifcount = ifcount - 1
            if (ifcount < level):
                level = level - 1
                if (level < 0):
                    print("Macro: Unmatched if/endif pairs.")
                    return None

        elif (args[0] == "display"):
            _vars['display'] = "on"

        else:
            if (runlevel[level] == True):
                if (result == ""):
                    result = subvars(line)
                else:
                    result = result + "\n" + subvars(line)
                    
    return(result)

### Substitute Vars
This routine is used by the runMacro program to track variables that are used within Macros. These are kept separate from the rest of the code.

In [ ]:
def subvars(script):
    
    global _vars    
    
    remainder = script
    result = ""
    done = False
    upper = False
    
    while done == False:
        bv = remainder.find("{")
        if (bv == -1):
            done = True
            continue
        ev = remainder.find("}")
        if (ev == -1):
            done = True
            continue
        result = result + remainder[:bv]
        vvar = remainder[bv+1:ev]
        remainder = remainder[ev+1:]

        if (vvar[0] == "^"):
            upper = True
            vvar = vvar[1:]
        else:
            upper = False
        
        if (vvar in _vars):
            if (upper == True):
                items = _vars[vvar].upper()
            else:
                items = _vars[vvar]
        else:
            items = "null"                
                 
        result = result + items
                
    if (remainder != ""):
        result = result + remainder
        
    return(result)

### Split Assignment
This routine will return the name of a variable and it's value when the format is x=y. If y is enclosed in quotes, the quotes are removed.

In [ ]:
def splitassign(arg):
    
    var_name = "null"
    var_value = "null"
    
    arg = arg.strip()
    eq = arg.find("=")
    if (eq != -1):
        var_name = arg[:eq].strip()
        temp_value = arg[eq+1:].strip()
        if (temp_value != ""):
            ch = temp_value[0]
            if (ch in ["'",'"']):
                if (temp_value[-1:] == ch):
                    var_value = temp_value[1:-1]
                else:
                    var_value = temp_value
            else:
                var_value = temp_value
    else:
        var_value = arg
            
    return var_name, var_value

### Parse Args 
The commands that are used in the macros need to be parsed into their separate components
There are three variables that get set:
- argc - The number of parameters
- argv - The entire string
- 0,1,2,3, - The arguments

In [ ]:
def parseArgs(argin):
    
    global _vars
    quoteChar = ""
    inQuote = False
    inArg = True
    args = []
    arg = ''
    
    for ch in argin:
        if (inQuote == True):
            if (ch == quoteChar):
                inQuote = False   
                arg = arg + ch #z
            else:
                arg = arg + ch
        elif (ch == "\"" or ch == "\'"): # Do we have a quote
            quoteChar = ch
            arg = arg + ch #z
            inQuote = True
        elif (ch == " "):
            if (arg != ""):
                arg = subvars(arg)
                args.append(arg)
            else:
                args.append("null")
            arg = ""
        else:
            arg = arg + ch
                
    if (arg != ""):
        arg = subvars(arg)
        args.append(arg)   
               
    return(args)

### SQL Parser

The calling format of this routine is:

```
sql_cmd, encoded_sql = sqlParser(sql_input)
```

This code will look at the SQL string that has been passed to it and parse it into two values:
- `sql_cmd`: First command in the list (so this may not be the actual SQL command)
- `encoded_sql`: SQL with the parameters removed and replaced with the real values

**Note:** The Variables must be "simple" in the sense that they are not lists or arrays. They can be dictionaries which are translated into JSON strings. If you want the contents of the variable to be enclosed in quotes, you need to supply them around the variable as in `':var'`.

In [ ]:
def sqlParser(sqlin):
       
    sql_cmd = ""
    remainder = ""
    encoded_sql = sqlin
    
    firstCommand = "(?:^\s*)([a-zA-Z]+)(?:\s+.*|$)"
    
    findFirst = re.match(firstCommand,sqlin)
    
    if (findFirst == None): # We did not find a match so we just return the empty string
        return sql_cmd, encoded_sql
    
    cmd = findFirst.group(1)
    sql_cmd = cmd.upper()

    #
    # Scan the input string looking for variables in the format :var. If no : is found just return.
    # Var must be alpha+number+_ to be valid
    #
    
    if (':' not in sqlin): # A quick check to see if parameters are in here, but not fool-proof!         
        return sql_cmd, encoded_sql    
    
    inVar = False 
    varName = ""
    encoded_sql = ""
    
    for ch in sqlin:
        if (inVar == True): # We are collecting the name of a variable
            if (ch.upper() in "_ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"):
                varName = varName + ch
                continue
            else:
                varValue = getContents(varName)
                if (varValue == None): 
                    varValue = ":" + varName
 
                encoded_sql = encoded_sql + varValue
                varName = ""
                inVar = False
               
        if (ch == ":"): # This might be a variable
            varName = ""
            inVar = True
        else:
            encoded_sql = encoded_sql + ch
        
    # We close a quoted string if you forgot it
    
    if (inVar == True):
        varValue = getContents(varName)
        if (varValue == None): varValue = ":" + varName
        encoded_sql = encoded_sql + varValue       
        
    return sql_cmd, encoded_sql

### Variable Contents Function
The calling format of this routine is:

```
value = getContents(varName)
```

This code will take the name of a variable as input and return the contents of that variable. If the variable is not found then the program will return None which is the equivalent to empty or null. Note that this function looks at the global variable pool for Python so it is possible that the wrong version of variable is returned if it is used in different functions. For this reason, any variables used in SQL statements should use a unique namimg convention if possible.

The other thing that this function does is replace single quotes with two quotes. The reason for doing this is that Db2 will convert two single quotes into one quote when dealing with strings. This avoids problems when dealing with text that contains multiple quotes within the string. Note that this substitution is done only for single quote characters since the double quote character is used by Db2 for naming columns that are case sensitive or contain special characters.

In [ ]:
def getContents(varName):
    
    #
    # Get the contents of the variable name that is passed to the routine. Only simple
    # variables are checked, i.e. arrays and lists are not parsed
    #
 
    varValue = None
    
    if (varName in globals()): # Does the variable exist?
        temp_global = globals().copy()
        tempValue = temp_global[varName]
        if (isinstance(tempValue,dict) == True):
            varValue = json.dumps(tempValue)
            # Need to change the \' escape characters to two quotes for Db2 processing
            varValue = varValue.replace("\'","''")
        else:
            # Need to replace single quotes with double quotes
            varValue = tempValue.replace("'","''")
    else:
        varValue = None

    return(varValue)

### Create the SAMPLE Database Tables
The calling format of this routine is:

```
db2_create_sample(quiet)
```

There are a lot of examples that depend on the data within the SAMPLE database. If you are running these examples and the connection is not to the SAMPLE database, then this code will create the two (EMPLOYEE, DEPARTMENT) tables that are used by most examples. If the function finds that these tables already exist, then nothing is done. If the tables are missing then they will be created with the same data as in the SAMPLE database.

The quiet flag tells the program not to print any messages when the creation of the tables is complete.

In [ ]:
def db2_create_sample(quiet):
    
    create_department = """
      BEGIN
        DECLARE FOUND INTEGER; 
        SET FOUND = (SELECT COUNT(*) FROM SYSIBM.SYSTABLES WHERE NAME='DEPARTMENT' AND CREATOR=CURRENT USER); 
        IF FOUND = 0 THEN 
           EXECUTE IMMEDIATE('CREATE TABLE DEPARTMENT(DEPTNO CHAR(3) NOT NULL, DEPTNAME VARCHAR(36) NOT NULL, 
                              MGRNO CHAR(6),ADMRDEPT CHAR(3) NOT NULL)'); 
           EXECUTE IMMEDIATE('INSERT INTO DEPARTMENT VALUES 
             (''A00'',''SPIFFY COMPUTER SERVICE DIV.'',''000010'',''A00''), 
             (''B01'',''PLANNING'',''000020'',''A00''), 
             (''C01'',''INFORMATION CENTER'',''000030'',''A00''), 
             (''D01'',''DEVELOPMENT CENTER'',NULL,''A00''), 
             (''D11'',''MANUFACTURING SYSTEMS'',''000060'',''D01''), 
             (''D21'',''ADMINISTRATION SYSTEMS'',''000070'',''D01''), 
             (''E01'',''SUPPORT SERVICES'',''000050'',''A00''), 
             (''E11'',''OPERATIONS'',''000090'',''E01''), 
             (''E21'',''SOFTWARE SUPPORT'',''000100'',''E01''), 
             (''F22'',''BRANCH OFFICE F2'',NULL,''E01''), 
             (''G22'',''BRANCH OFFICE G2'',NULL,''E01''), 
             (''H22'',''BRANCH OFFICE H2'',NULL,''E01''), 
             (''I22'',''BRANCH OFFICE I2'',NULL,''E01''), 
             (''J22'',''BRANCH OFFICE J2'',NULL,''E01'')');      
           END IF;
      END"""
  
    %sql -d -q {create_department} 
    
    create_employee = """
     BEGIN
        DECLARE FOUND INTEGER; 
        SET FOUND = (SELECT COUNT(*) FROM SYSIBM.SYSTABLES WHERE NAME='EMPLOYEE' AND CREATOR=CURRENT USER); 
        IF FOUND = 0 THEN 
          EXECUTE IMMEDIATE('CREATE TABLE EMPLOYEE(
                             EMPNO CHAR(6) NOT NULL,
                             FIRSTNME VARCHAR(12) NOT NULL,
                             MIDINIT CHAR(1),
                             LASTNAME VARCHAR(15) NOT NULL,
                             WORKDEPT CHAR(3),
                             PHONENO CHAR(4),
                             HIREDATE DATE,
                             JOB CHAR(8),
                             EDLEVEL SMALLINT NOT NULL,
                             SEX CHAR(1),
                             BIRTHDATE DATE,
                             SALARY DECIMAL(9,2),
                             BONUS DECIMAL(9,2),
                             COMM DECIMAL(9,2)
                             )');
          EXECUTE IMMEDIATE('INSERT INTO EMPLOYEE VALUES
             (''000010'',''CHRISTINE'',''I'',''HAAS''      ,''A00'',''3978'',''1995-01-01'',''PRES    '',18,''F'',''1963-08-24'',152750.00,1000.00,4220.00),
             (''000020'',''MICHAEL''  ,''L'',''THOMPSON''  ,''B01'',''3476'',''2003-10-10'',''MANAGER '',18,''M'',''1978-02-02'',94250.00,800.00,3300.00),
             (''000030'',''SALLY''    ,''A'',''KWAN''      ,''C01'',''4738'',''2005-04-05'',''MANAGER '',20,''F'',''1971-05-11'',98250.00,800.00,3060.00),
             (''000050'',''JOHN''     ,''B'',''GEYER''     ,''E01'',''6789'',''1979-08-17'',''MANAGER '',16,''M'',''1955-09-15'',80175.00,800.00,3214.00),
             (''000060'',''IRVING''   ,''F'',''STERN''     ,''D11'',''6423'',''2003-09-14'',''MANAGER '',16,''M'',''1975-07-07'',72250.00,500.00,2580.00),
             (''000070'',''EVA''      ,''D'',''PULASKI''   ,''D21'',''7831'',''2005-09-30'',''MANAGER '',16,''F'',''2003-05-26'',96170.00,700.00,2893.00),
             (''000090'',''EILEEN''   ,''W'',''HENDERSON'' ,''E11'',''5498'',''2000-08-15'',''MANAGER '',16,''F'',''1971-05-15'',89750.00,600.00,2380.00),
             (''000100'',''THEODORE'' ,''Q'',''SPENSER''   ,''E21'',''0972'',''2000-06-19'',''MANAGER '',14,''M'',''1980-12-18'',86150.00,500.00,2092.00),
             (''000110'',''VINCENZO'' ,''G'',''LUCCHESSI'' ,''A00'',''3490'',''1988-05-16'',''SALESREP'',19,''M'',''1959-11-05'',66500.00,900.00,3720.00),
             (''000120'',''SEAN''     ,'' '',''O`CONNELL'' ,''A00'',''2167'',''1993-12-05'',''CLERK   '',14,''M'',''1972-10-18'',49250.00,600.00,2340.00),
             (''000130'',''DELORES''  ,''M'',''QUINTANA''  ,''C01'',''4578'',''2001-07-28'',''ANALYST '',16,''F'',''1955-09-15'',73800.00,500.00,1904.00),
             (''000140'',''HEATHER''  ,''A'',''NICHOLLS''  ,''C01'',''1793'',''2006-12-15'',''ANALYST '',18,''F'',''1976-01-19'',68420.00,600.00,2274.00),
             (''000150'',''BRUCE''    ,'' '',''ADAMSON''   ,''D11'',''4510'',''2002-02-12'',''DESIGNER'',16,''M'',''1977-05-17'',55280.00,500.00,2022.00),
             (''000160'',''ELIZABETH'',''R'',''PIANKA''    ,''D11'',''3782'',''2006-10-11'',''DESIGNER'',17,''F'',''1980-04-12'',62250.00,400.00,1780.00),
             (''000170'',''MASATOSHI'',''J'',''YOSHIMURA'' ,''D11'',''2890'',''1999-09-15'',''DESIGNER'',16,''M'',''1981-01-05'',44680.00,500.00,1974.00),
             (''000180'',''MARILYN''  ,''S'',''SCOUTTEN''  ,''D11'',''1682'',''2003-07-07'',''DESIGNER'',17,''F'',''1979-02-21'',51340.00,500.00,1707.00),
             (''000190'',''JAMES''    ,''H'',''WALKER''    ,''D11'',''2986'',''2004-07-26'',''DESIGNER'',16,''M'',''1982-06-25'',50450.00,400.00,1636.00),
             (''000200'',''DAVID''    ,'' '',''BROWN''     ,''D11'',''4501'',''2002-03-03'',''DESIGNER'',16,''M'',''1971-05-29'',57740.00,600.00,2217.00),
             (''000210'',''WILLIAM''  ,''T'',''JONES''     ,''D11'',''0942'',''1998-04-11'',''DESIGNER'',17,''M'',''2003-02-23'',68270.00,400.00,1462.00),
             (''000220'',''JENNIFER'' ,''K'',''LUTZ''      ,''D11'',''0672'',''1998-08-29'',''DESIGNER'',18,''F'',''1978-03-19'',49840.00,600.00,2387.00),
             (''000230'',''JAMES''    ,''J'',''JEFFERSON'' ,''D21'',''2094'',''1996-11-21'',''CLERK   '',14,''M'',''1980-05-30'',42180.00,400.00,1774.00),
             (''000240'',''SALVATORE'',''M'',''MARINO''    ,''D21'',''3780'',''2004-12-05'',''CLERK   '',17,''M'',''2002-03-31'',48760.00,600.00,2301.00),
             (''000250'',''DANIEL''   ,''S'',''SMITH''     ,''D21'',''0961'',''1999-10-30'',''CLERK   '',15,''M'',''1969-11-12'',49180.00,400.00,1534.00),
             (''000260'',''SYBIL''    ,''P'',''JOHNSON''   ,''D21'',''8953'',''2005-09-11'',''CLERK   '',16,''F'',''1976-10-05'',47250.00,300.00,1380.00),
             (''000270'',''MARIA''    ,''L'',''PEREZ''     ,''D21'',''9001'',''2006-09-30'',''CLERK   '',15,''F'',''2003-05-26'',37380.00,500.00,2190.00),
             (''000280'',''ETHEL''    ,''R'',''SCHNEIDER'' ,''E11'',''8997'',''1997-03-24'',''OPERATOR'',17,''F'',''1976-03-28'',36250.00,500.00,2100.00),
             (''000290'',''JOHN''     ,''R'',''PARKER''    ,''E11'',''4502'',''2006-05-30'',''OPERATOR'',12,''M'',''1985-07-09'',35340.00,300.00,1227.00),
             (''000300'',''PHILIP''   ,''X'',''SMITH''     ,''E11'',''2095'',''2002-06-19'',''OPERATOR'',14,''M'',''1976-10-27'',37750.00,400.00,1420.00),
             (''000310'',''MAUDE''    ,''F'',''SETRIGHT''  ,''E11'',''3332'',''1994-09-12'',''OPERATOR'',12,''F'',''1961-04-21'',35900.00,300.00,1272.00),
             (''000320'',''RAMLAL''   ,''V'',''MEHTA''     ,''E21'',''9990'',''1995-07-07'',''FIELDREP'',16,''M'',''1962-08-11'',39950.00,400.00,1596.00),
             (''000330'',''WING''     ,'' '',''LEE''       ,''E21'',''2103'',''2006-02-23'',''FIELDREP'',14,''M'',''1971-07-18'',45370.00,500.00,2030.00),
             (''000340'',''JASON''    ,''R'',''GOUNOT''    ,''E21'',''5698'',''1977-05-05'',''FIELDREP'',16,''M'',''1956-05-17'',43840.00,500.00,1907.00),
             (''200010'',''DIAN''     ,''J'',''HEMMINGER'' ,''A00'',''3978'',''1995-01-01'',''SALESREP'',18,''F'',''1973-08-14'',46500.00,1000.00,4220.00),
             (''200120'',''GREG''     ,'' '',''ORLANDO''   ,''A00'',''2167'',''2002-05-05'',''CLERK   '',14,''M'',''1972-10-18'',39250.00,600.00,2340.00),
             (''200140'',''KIM''      ,''N'',''NATZ''      ,''C01'',''1793'',''2006-12-15'',''ANALYST '',18,''F'',''1976-01-19'',68420.00,600.00,2274.00),
             (''200170'',''KIYOSHI''  ,'' '',''YAMAMOTO''  ,''D11'',''2890'',''2005-09-15'',''DESIGNER'',16,''M'',''1981-01-05'',64680.00,500.00,1974.00),
             (''200220'',''REBA''     ,''K'',''JOHN''      ,''D11'',''0672'',''2005-08-29'',''DESIGNER'',18,''F'',''1978-03-19'',69840.00,600.00,2387.00),
             (''200240'',''ROBERT''   ,''M'',''MONTEVERDE'',''D21'',''3780'',''2004-12-05'',''CLERK   '',17,''M'',''1984-03-31'',37760.00,600.00,2301.00),
             (''200280'',''EILEEN''   ,''R'',''SCHWARTZ''  ,''E11'',''8997'',''1997-03-24'',''OPERATOR'',17,''F'',''1966-03-28'',46250.00,500.00,2100.00),
             (''200310'',''MICHELLE'' ,''F'',''SPRINGER''  ,''E11'',''3332'',''1994-09-12'',''OPERATOR'',12,''F'',''1961-04-21'',35900.00,300.00,1272.00),
             (''200330'',''HELENA''   ,'' '',''WONG''      ,''E21'',''2103'',''2006-02-23'',''FIELDREP'',14,''F'',''1971-07-18'',35370.00,500.00,2030.00),
             (''200340'',''ROY''      ,''R'',''ALONZO''    ,''E21'',''5698'',''1997-07-05'',''FIELDREP'',16,''M'',''1956-05-17'',31840.00,500.00,1907.00)');                             
        END IF;
     END"""
    
    %sql -d -q {create_employee}    
    
    if (quiet == False): success("Sample tables [EMPLOYEE, DEPARTMENT] created.")

### Check option
This function will return the original string with the option removed, and a flag or true or false of the value is found.

```
args, flag = checkOption(option_string, option, false_value, true_value)
```

Options are specified with a -x where x is the character that we are searching for. It may actually be more than one character long like -pb/-pi/etc... The false and true values are optional. By default these are the boolean values of T/F but for some options it could be a character string like ';' versus '@' for delimiters.

In [ ]:
def checkOption(args_in, option, vFalse=False, vTrue=True):
    
    args_out = args_in.strip()
    found = vFalse
    
    if (args_out != ""):
        if (args_out.find(option) >= 0):
            args_out = args_out.replace(option," ")
            args_out = args_out.strip()
            found = vTrue

    return args_out, found

### Plot Data
This function will plot the data that is returned from the answer set. The plot value determines how we display the data. 1=Bar, 2=Pie, 3=Line, 4=Interactive.

```
plotData(flag_plot, hdbi, sql, parms)
```

The hdbi is the ibm_db_sa handle that is used by pandas dataframes to run the sql. The parms contains any of the parameters required to run the query.

In [ ]:
def plotData(df):
    
    if df.empty:
        errormsg("No results returned")
        return
    
    if flag("-i"):                                    # Plot 4 = pixiedust

        ShellAccess.pdf = df
        display(pdf)
        return
    
    col_count = len(df.columns)

    if flag("-pb"):                                    # Plot 1 = bar chart
    
        if (col_count in (1,2,3)):
             
            if (col_count == 1):
 
                df.index = df.index + 1
                _ = df.plot(kind='bar');
                
            elif (col_count == 2):
 
                xlabel = df.columns.values[0]
                ylabel = df.columns.values[1]
                _ = df.plot(kind='bar',x=xlabel,y=ylabel);

            else:
 
                values = df.columns.values[2]
                columns = df.columns.values[0]
                index = df.columns.values[1]
                pivoted = pandas.pivot_table(df, values=values, columns=columns, index=index) 
                _ = pivoted.plot.bar(); 
                
            plt.show()
            
        else:
            errormsg("Can't determine what columns to plot")
            return
                    
    elif flag("-pp"):                                  # Plot 2 = pie chart
        
        if (col_count in (1,2)):
                
            if (col_count == 1):
                df.index = df.index + 1
                yname = df.columns.values[0]
                _ = df.plot(kind='pie',y=yname);                
            else:          
                xlabel = df.columns.values[0]
                xname = df[xlabel].tolist()
                yname = df.columns.values[1]
                _ = df.plot(kind='pie',y=yname,labels=xname);
                
            plt.show();
            
        else:
            errormsg("Can't determine what columns to plot")
            return
                    
    elif flag("-pl"):                                  # Plot 3 = line chart
            
        if (col_count in (1,2,3)):
            
            if (col_count == 1):
                df.index = df.index + 1  
                _ = df.plot(kind='line');          
            elif (col_count == 2):            
                xlabel = df.columns.values[0]
                ylabel = df.columns.values[1]
                _ = df.plot(kind='line',x=xlabel,y=ylabel) ; 
            else:         
                values = df.columns.values[2]
                columns = df.columns.values[0]
                index = df.columns.values[1]
                pivoted = pandas.pivot_table(df, values=values, columns=columns, index=index)
                _ = pivoted.plot();
                
            plt.show();
                
        else:
            errormsg("Can't determine what columns to plot")
            return
    else:
        return

### Set Flags
This code will take the input SQL block and update the global flag list. The global flag list is just a list of options that are set at the beginning of a code block. The absence of a flag means it is false. If it exists it is true.

In [ ]:
def setFlags(inSQL):
    
    global _flags
    
    _flags = [] # Delete all of the current flag settings
    
    pos = 0
    end = len(inSQL)-1
    inFlag = False
    ignore = False
    outSQL = ""
    flag = ""
    
    while (pos <= end):
        ch = inSQL[pos]
        if (ignore == True):   
            outSQL = outSQL + ch
        else:
            if (inFlag == True):
                if (ch != " "):
                    flag = flag + ch
                else:
                    _flags.append(flag)
                    inFlag = False
            else:
                if (ch == "-"):
                    flag = "-"
                    inFlag = True
                elif (ch == ' '):
                    outSQL = outSQL + ch
                else:
                    outSQL = outSQL + ch
                    ignore = True
        pos += 1
        
    if (inFlag == True):
        _flags.append(flag)
        
    return outSQL.strip()

### Check to see if flag Exists
This function determines whether or not a flag exists in the global flag array. Absence of a value means it is false. The parameter can be a single value, or an array of values.

In [ ]:
def flag(inflag):
    
    global _flags

    if isinstance(inflag,list):
        for x in inflag:
            if (x in _flags):
                return True
        return False
    else:
        if (inflag in _flags):
            return True
        else:
            return False

### Submit an SQL Job
The submitSQL call is used to send a RESTful request to Db2 with the SQL you want executed. The function will return a job id if the call is successful, otherwise it will return None. The function only needs the SQL and the delimiter used for the SQL that is executed.

One of the issues when using access tokens is that they expire after a certain period of time. This makes the access more secure since a token will expire in the event that it was accidentially released externally. However, this means that your queries may not work after a period of time because of an expired access token. This routine attempts to generate a new access token in the event that the call fails.

In [ ]:
def submitSQL(sql,separator=";"):
    
    global _settings
    
    db2id = _settings["id"]
    api = _settings["api"]
    token = _settings["token"]
    host = db2id["https_url"] + api
    
    # Check to see if we need to get an access token
    
    if (token == None):
        token = get_token()
        if (token == None): return None
        _settings["token"] = token
        
    service = "/sql_jobs"     

    # Request to send 
    
    request = {
        "commands" : sql,
        "limit" : 10000,
        "separator" : separator,
        "stop_on_error" : "no"
    }
    
    connected = False
    
    while connected == False:         
        
        authid = {"Authorization" : "Bearer " + token}
        
        try:
            r = requests.post(host + service,headers=authid,json=request)
            if (r.ok == False): # Some error occured on the post request
                json_result = r.json()
                error_call = json_result['errors'][0]
                if (error_call['code'] == 'authentication_failure'):
                    token = get_token()
                    if (token == None): return None
                    _settings['token'] = token
                    retry = True
                    continue;
                else:
                    print("SQL Execution error")
                    print(json_result)
                    return None
            else:
                connected = True
                continue
                
        except Exception as err:
        
            print("Error attempting RESTful call")
            print(err)
            return None
        
    # Retrieve the ID of the job that is running and return it to the calling program
    
    jobid = r.json()['id']
    
    return jobid

### Retrieve Results

The function has the following format:
```
retrieve(jobid)
```
The function will return a return code and a pandas dataframe or an array as a result set. The job id is required as the only parameter.

The code in this function also checks to make sure that the SQL has completed execution. There is a possibility that the SQL will run for a long period of time so the function needs to continuously poll the server with RESTful calls until the SQL has completed.

Return format: (code,results)

The return code can be:

* 0 = SQL executed successfully with an answer set
* 1 = Error in executing the RESTful call
* 2 = SQL error, but can continue running
* 3 - SQL executed successfully command

In [ ]:
def retrieveSQL(jobid):

    global _settings
    
    db2id = _settings["id"]
    api = _settings["api"]
    token = _settings["token"]
    host = db2id["https_url"] + api    

    service = "/sql_jobs/" + jobid
    authid = {"Authorization" : "Bearer " + token}
    
    flag_message = False
    
    rows = None
    
    while True:
        try:
            
            r = requests.get(host + service,headers=authid)
            if (r.ok == False):
                print("SQL request failed. Check authentication")
                print(r.json())
                return 1,None
            
        except Exception as err:
            
            print("Error attempting RESTful call")
            print(err)
            return 1,None
        
        try:
            
            json_returned = r.json()
            status = json_returned['status']
            results = json_returned['results']
            
        except Exception as err:
 
            print("Unable to retrieve status information from RESTful call.")
            print(json_returned)
            return 1,None
        
        if (status == "failed"):
            
            try:
                error_text = results[0]['error']
                if (flag("-q") == False): 
                    errormsg(error_text)
                return 2,None
                    
            except Exception as err:

                print("SQL failed. Unable to retrieve reason code.")
                print(json_returned)
                return 1,None
                
            return False
        
        elif (status == "running" and len(results) == 0):
            
            continue
            
        else:
            
            pass
        
        try:
            for result in results:
                columns = result['columns']
                row_count = result['rows_count']
                error_msg = result['error']
                if error_msg != "":
                    if (flag("-q") == False):
                        errormsg(error_msg)
                        flag_message = True
                else:
                    if len(columns) > 0:
                        if row_count == 0: return 4,None
                        rows = result['rows']
                        if (flag(["-r","-j"]) == True and flag(["-pb","-pp","-pl","-i"]) == False): 
                            # We are plotting some results 
                            rows.insert(0,columns)
                            return 0,rows
                        else:
                            df = pandas.DataFrame.from_records(data=rows,columns=columns,coerce_float=True)
                            cols = df.columns
                            df[cols] = df[cols].apply(pandas.to_numeric, errors='ignore')
                            return 0,df
                    else:
                        continue
                        
        except Exception as err:
            pass
            
        # Keep trying the loop until our job is complete    
        if (status == 'completed'): break
    
    if (flag_message == True):
        return 3,None
    else:
        return 2,None


### Main %sql Magic Definition
The main %sql Magic logic is found in this section of code. This code will register the Magic command and allow Jupyter notebooks to interact with Db2 by using this extension.


In [ ]:
@magics_class
class DB2(Magics):
        
    @line_cell_magic
    def sql(self, line, cell=None):
    
        global _settings, sqlstate, sqlerror, sqlcode
        
        import qgrid
           
        # If you use %sql (line) we just run the SQL. If you use %%SQL the entire cell is run.
        
        flag_cell = False
        flag_output = False
        sqlstate = "0"
        sqlerror = ""
        sqlcode = 0        
              
        # Macros gets expanded before anything is done
                
        SQL1 = setFlags(line.strip())  
        SQL2 = cell
        
        if flag("-sampledata"):                                      # Check if you only want sample data loaded
            if (_settings['connected'] == False):
                errormsg('A CONNECT statement must be issued before creating the sample tables.')
                return
            db2_create_sample(flag("-q"))
            return 
        
        if len(SQL1) == 0 and SQL2 == None: return     # Nothing to do here
        
        if SQL1 == "?" or flag("-h"):                             # Are you asking for help
            sqlhelp()
            return
        
        if SQL1.upper() == "? CONNECT":                       # Are you asking for help on CONNECT
            connected_help()
            return        
        
        sqlType,remainder = sqlParser(SQL1)                          # If this is a CONNECT command, run it alone
 
        if (sqlType == "CONNECT"):
            parseConnect(SQL1)
            return
        elif (sqlType == "MACRO"): 
            result = setMacro(SQL2,remainder)
            return
        else:
            pass        
        
        if (SQL2 != None): sqlType,SQL2 = sqlParser(SQL2)
 
        if (SQL2 != None):
            SQL2 = searchMacro(SQL2)                                       # Macros in a cell
            if (SQL2 == None): return                
            runSQL = SQL2
        else:
            SQL1 = searchMacro(remainder)                                   # Run any macros in the code
            if (SQL1 == None): return       
            runSQL = SQL1
        
        if (_settings['connected'] == False):
            errormsg('A CONNECT statement must be issued before running any SQL commands.')
            return
                  
        runSQL = re.sub('.*?--.*$',"",runSQL,flags=re.M)
        remainder = runSQL.replace("\n"," ")        

        if flag("-d"):                                            # Run the SQL and check the results
            jobid = submitSQL(remainder,"@")
        else:
            jobid = submitSQL(remainder,";")
            
        if (jobid == None): return None

        rc, results = retrieveSQL(jobid)
        
        if (rc == 2) : 
            if (flag("-q") == False): print("Command completed")
            return
        elif (rc == 4) :
            print("No results found.")
            return
        elif (rc == 1 or rc == 3):
            return
        else:
            pass

        if flag("-r"):                                 # Raw output required
            return(results)
        elif flag(["-pb","-pp","-pl","-i"]):                  # We are plotting some results 
            plotData(results)                           # Plot the data and return
            return
        elif flag("-j"):                                 # JSON output
            row_count = 0
            json_results = []
            for row in results:
                row_count += 1
                if (row_count == 1): continue          # Skip the column names
                jsonDict = json.loads(row[0])          # Get the first column of the answer set
                json_results.append(jsonDict)                               
                return(json_results)  
            
        else:
            if flag("-a"):   # Check to see if we can display the results
                pdisplay(qgrid.show_grid(results))
            else:
                return results
            
# Register the Magic extension in Jupyter    
ip = get_ipython()          
ip.register_magics(DB2)
   
success("DB2 Extensions Loaded.")

Turning off Pixiedust data collection.

In [ ]:
%%capture
try: 
    _ = pixiedust.optOut()
except:
    None

Set the table formatting to left align a table in a cell. By default, tables are centered in a cell. Remove this cell if you don't want to change Jupyter notebook formatting for tables.

In [ ]:
%%html
<style>
  table {margin-left: 0 !important; text-align: left;}
</style>

#### Credits: IBM 2018, George Baklarz [baklarz@ca.ibm.com]